In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import numpy as np
import os
from xml.etree import ElementTree
import numpy as np
import math as m
from scipy import io
import re

In [ ]:
from ipynb.fs.full.XMLRead import *

In [ ]:
def FailureMode(ip1sig,ip1eps,trussSig1,trussSig2,trussSig3,trussEps1,trussEps2,trussEps3,refFile):
    with open(refFile, 'rt') as h:
        data = [line.split() for line in h]
        nodes = int(data[0][0])
        trussNodes = int(data[1][0])
        name = (data[7][0])
        filepath = data[8][0]
        
    loaded = io.loadmat('WallData.mat')
    data = loaded['WallData']
    loc = data[0,0]["UniqueID"]

    for ii in range(len(loc)):
        if name == loc[ii][0][0]:
            index = ii
            break

    fc = -5.28074
    eles = nodes - 1
    conSigL1,conSigL2,conSigL3,conSigR1,conSigR2,conSigR3 = [],[],[],[],[],[]
    conEpsL1,conEpsL2,conEpsL3,conEpsR1,conEpsR2,conEpsR3 = [],[],[],[],[],[]
    
    minLen = min([len(ip1sig),len(ip1eps)])
    
    for ii in range(minLen):
        conSigL1.append(ip1sig[ii][1])
        conEpsL1.append(ip1eps[ii][1])
        conSigL2.append(ip1sig[ii][eles*5+1])
        conEpsL2.append(ip1eps[ii][eles*5+1])    
        conSigL3.append(ip1sig[ii][eles*2*5+1])
        conEpsL3.append(ip1eps[ii][eles*2*5+1])  
        conSigR1.append(ip1sig[ii][(eles-1)*5+1])
        conEpsR1.append(ip1eps[ii][(eles-1)*5+1])
        conSigR2.append(ip1sig[ii][(eles*2-1)*5+1])
        conEpsR2.append(ip1eps[ii][(eles*2-1)*5+1])    
        conSigR3.append(ip1sig[ii][(eles*3-1)*5+1])
        conEpsR3.append(ip1eps[ii][(eles*3-1)*5+1])  
    SteelSigL1,SteelSigL2,SteelSigL3,SteelSigR1,SteelSigR2,SteelSigR3 = [],[],[],[],[],[]
    SteelEpsL1,SteelEpsL2,SteelEpsL3,SteelEpsR1,SteelEpsR2,SteelEpsR3 = [],[],[],[],[],[]
    

    for ii in range(len(trussSig1)):
        SteelSigL1.append(trussSig1[ii][0])
        SteelEpsL1.append(trussEps1[ii][0])
        SteelSigL2.append(trussSig2[ii][0])
        SteelEpsL2.append(trussEps2[ii][0])    
        SteelSigL3.append(trussSig3[ii][0])
        SteelEpsL3.append(trussEps3[ii][0])
        SteelSigR1.append(trussSig1[ii][-1])
        SteelEpsR1.append(trussEps1[ii][-1])
        SteelSigR2.append(trussSig2[ii][-1])
        SteelEpsR2.append(trussEps2[ii][-1])     
        SteelSigR3.append(trussSig3[ii][-1])
        SteelEpsR3.append(trussEps3[ii][-1])
        
    
    ind = []
    files = [SteelSigL1,SteelSigL2,SteelSigL3,SteelSigR1,SteelSigR2,SteelSigR3,conSigL1,conSigL2,conSigL3,conSigR1,conSigR2,conSigR3]
    epsFiles = [SteelEpsL1,SteelEpsL2,SteelEpsL3,SteelEpsR1,SteelEpsR2,SteelEpsR3,conEpsL1,conEpsL2,conEpsL3,conEpsR1,conEpsR2,conEpsR3]
    print(fc)
    for jj in range(len(files)):
        fileSig = files[jj]
        fileEps = epsFiles[jj]
        pastFc = False
        for ii in range(1,len(fileSig)):
            if jj> 5:
                #if m.isclose(fileSig[ii],fc,abs_tol = 0.1):
                #    pastFc = True
                #    print(fileSig[ii])
                    
                if fileEps[ii] < -0.01 and abs(fileSig[ii]) <0.3*abs(fc):
                    print(fileSig[ii])
                    ind.append(ii)
                    break
                    
            elif jj<6 and m.isclose(fileEps[ii],0.044,abs_tol = 0.001):
                ind.append(ii)
                break
                
            elif ii == len(files[jj])-1:
                ind.append(0)
                break
    
    totind = 0
    for ii in range(len(ind)):
        totind += ind[ii]
        
    if totind == 0:
        FM = 'None'
    else:
        maxBreak = ind[-1]
        brk = 0
        for ii in range(len(ind)):
            if ind[ii] < maxBreak:
                brk = ii

        if ii < 6:
            FM = 'BR'
        else:
            FM = 'CB'
    fig, ax = plt.subplots(2,2,figsize=(12, 10),linewidth=4)
    #fig.suptitle(FM,fontsize=24)
    ax[0,0].plot(conEpsL1,conSigL1,  label = 'Extreme Left Side Level 0') 
    ax[0,0].plot(conEpsL2,conSigL2,  label = 'Extreme Left Side Level 1') 
    ax[0,0].plot(conEpsL3,conSigL3,  label = 'Extreme Left Side Level 2') 
    ax[0,0].set_ylabel('Stress (ksi)',fontsize=12)
    ax[0,0].set_xlabel('Strain (in/in)',fontsize=12)
    ax[0,0].legend(loc = 'lower right',fontsize=12)
    ax[0,0].grid(True)
    ax[0,0].set_title('Concrete Left Side',fontsize=20)
    
    
    ax[0,1].plot(conEpsR1,conSigR1,label = 'Extreme Right Side Level 0')
    ax[0,1].plot(conEpsR2,conSigR2,label = 'Extreme Right Side Level 1')
    ax[0,1].plot(conEpsR3,conSigR3,label = 'Extreme Right Side Level 2')  
    ax[0,1].set_ylabel('Stress (ksi)',fontsize=12)
    ax[0,1].set_xlabel('Strain (in/in)',fontsize=12)
    ax[0,1].legend(loc = 'lower right',fontsize=12)
    ax[0,1].grid(True)
    ax[0,1].set_title('Concrete Right Side',fontsize=20)
    

    ax[1,0].plot(SteelEpsL1,SteelSigL1,  label = 'Extreme Left Side Level 0') 
    ax[1,0].plot(SteelEpsL2,SteelSigL2,  label = 'Extreme Left Side Level 1') 
    ax[1,0].plot(SteelEpsL3,SteelSigL3,  label = 'Extreme Left Side Level 2') 
    ax[1,0].set_ylabel('Stress (ksi)',fontsize=12)
    ax[1,0].set_xlabel('Strain (in/in)',fontsize=12)
    ax[1,0].legend(loc = 'lower right',fontsize=12)
    ax[1,0].grid(True)
    ax[1,0].set_title('Steel Left Side',fontsize=20)
    
    ax[1,1].plot(SteelEpsR1,SteelSigR1,label = 'Extreme Right Side Level 0')
    ax[1,1].plot(SteelEpsR2,SteelSigR2,label = 'Extreme Right Side Level 1')
    ax[1,1].plot(SteelEpsR3,SteelSigR3,label = 'Extreme Right Side Level 2')  
    ax[1,1].set_ylabel('Stress (ksi)',fontsize=12)
    ax[1,1].set_xlabel('Strain (in/in)',fontsize=12)
    ax[1,1].legend(loc = 'lower right',fontsize=12)
    ax[1,1].grid(True)
    ax[1,1].set_title('Steel Right Side',fontsize=20)
    
    
    print(filepath+'FM.png')
    #plt.close(fig)
    fig.tight_layout()
    fig.savefig(filepath+'FM.png')
    return FM

In [ ]:
files = [
'tcl_files\\tcl_files_22110420\\WallaceRW1_221104203419\\221104203419referenceFile.txt',
'tcl_files\\tcl_files_22110420\\TranS38_221104203422\\221104203422referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ZhangWangSW9_221104203425\\221104203425referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ZhangWangSW8_230305015210\\230305015210referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ShegayA14_230305015207\\230305015207referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ShegayA20_230305015208\\230305015208referenceFile.txt',
'tcl_files\\tcl_files_22110420\\TranS63_230305015209\\230305015209referenceFile.txt',
'tcl_files\\tcl_files_22110420\\SeguraWP6_230301204151\\230301204151referenceFile.txt',
'tcl_files\\tcl_files_22110420\\OhWR10_230305015206\\230305015206referenceFile.txt',
'tcl_files\\tcl_files_22110420\\DazioWSH2_230305015050\\230305015050referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ZhangWangSW7_230211125344\\230211125344referenceFile.txt',
'tcl_files\\tcl_files_22110420\\ShegayC10_230211125342\\230211125342referenceFile.txt'
]

for ii in files:
    xmlFiles = XMLRead(ii)
    FailureMode(xmlFiles.ip1filesig,xmlFiles.ip1fileeps,xmlFiles.trussSig1,xmlFiles.trussSig2,xmlFiles.trussSig3,xmlFiles.trussEps1,xmlFiles.trussEps2,xmlFiles.trussEps3,ii)